In [1]:
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import skimage
import pandas as pd
import anndata as ad
from collections import namedtuple
import networkx as nx
import math
import sys
import getopt
import anndata as ad
from voronoi import *
import pickle


In [2]:
#class snakemake():
 #   input = ['/gpfs/commons/home/jsingh/results/test_anndata.h5ad', '/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/seurat_analysis/pyseq_test_types.csv'
  #          ,'/gpfs/commons/home/jsingh/results/uns.csv','/gpfs/commons/home/jsingh/results/centroid.npy']
   # 
    

In [3]:
class snakemake():
    
    input = ['/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/m1a.h5ad','/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/m1a.csv']
    output = ['/gpfs/commons/home/jsingh/results/neighbor_dict.pkl','/gpfs/commons/home/jsingh/results/cell_type_per_node.pkl']

In [4]:
adata = ad.read(snakemake.input[0])

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [5]:
x_coord = adata.X[:,-2]
y_coord = adata.X[:,-1]

Point = namedtuple('Point', ['x','y'])
point_list = []
for i,y in zip(x_coord,y_coord):
    point_list.append(Point(i,y))
v,l,e = computeVoronoiDiagram(point_list)

#Building Graph
gr = nx.Graph()
for pt in v:
    gr.add_node(pt)
    
edge_list= []
for ei in range(len(e)):
    edge_list.append(e[ei][1:])
    
for ed in edge_list:
    gr.add_edge(ed[0],ed[1])

In [6]:
e

[(4, 0, 1),
 (2, 1, 2),
 (3, 3, 0),
 (7, 4, 2),
 (5, 1, 6),
 (13, 5, 7),
 (9, 7, 3),
 (15, 6, 8),
 (10, 8, 4),
 (16, 7, 10),
 (20, 9, 12),
 (22, 12, 10),
 (25, 11, 13),
 (12, 13, 5),
 (27, 13, 14),
 (17, 14, 9),
 (24, 15, 11),
 (18, 8, 16),
 (11, 4, 17),
 (36, 18, 17),
 (26, 19, 12),
 (41, 20, 21),
 (38, 22, 19),
 (39, 21, 23),
 (35, 17, 23),
 (21, 10, 24),
 (14, 24, 6),
 (31, 16, 25),
 (33, 25, 18),
 (48, 24, 26),
 (28, 14, 27),
 (52, 27, 28),
 (19, 28, 9),
 (29, 15, 29),
 (53, 29, 27),
 (54, 28, 30),
 (44, 30, 22),
 (45, 22, 31),
 (37, 32, 18),
 (40, 32, 20),
 (46, 26, 33),
 (32, 33, 16),
 (50, 34, 25),
 (57, 31, 35),
 (34, 19, 35),
 (60, 33, 36),
 (49, 36, 34),
 (59, 37, 32),
 (64, 35, 38),
 (51, 38, 26),
 (69, 38, 39),
 (70, 39, 40),
 (72, 40, 41),
 (66, 42, 39),
 (63, 34, 43),
 (76, 43, 44),
 (62, 44, 37),
 (58, 45, 31),
 (71, 45, 42),
 (61, 41, 46),
 (81, 46, 47),
 (65, 47, 36),
 (56, 30, 48),
 (78, 48, 45),
 (43, 21, 49),
 (83, 50, 48),
 (75, 42, 51),
 (91, 52, 51),
 (67, 37, 53

In [7]:
df = pd.read_csv(snakemake.input[1])
cell_type = {}
for i,ty in zip(df.id,df.type):
    cell_type.update({i: ty})

In [8]:
df

,id,type
0,1,Oligodendrocytes
1,2,Oligodendrocytes
2,3,Inhibitory neurons
3,4,Excitatory neurons
4,5,Cholinergic neurons
...,...,...
108,109,Astrocytes
109,110,Astrocytes
110,111,Inhibitory neurons
111,112,Astrocytes


In [9]:
def get_neighbors(graph, node_val, rad):
    subgraph = nx.ego_graph(G = graph, n= node_val, radius=rad, center=True, undirected=False, distance=None)
    node_list = list(subgraph.nodes())
    return(node_list)
    

def get_cell_type(node_list):
    cell_frame = pd.DataFrame()
    cell_list = []
    final_node_list = []
    try:
        node_list.remove(0)
    except: 
        pass
    try:
        node_list.remove(-1)
    except:
        pass
    for n in node_list:
        try:
            cell_list.append(cell_type[n])
            final_node_list.append(n)
            
        except:
            node_list.remove(n)
            pass
    
    cell_frame['id'] = final_node_list
    cell_frame['type'] = cell_list
   
    return cell_frame 
        

def cell_pixel_intensity_gfap(frame):
    intensity_list = []
    for idx in frame.id:
        intensity_list.append(mean_intensities[idx - 1]) #minus 1 as we removed intensity for label 0
        
    frame['gfap_intensity'] = intensity_list
    
    
    return frame
        
        
    


In [10]:
def get_nodes(dt):
    selected_nodes = []
    for k in dt.keys():
        if dt[k] == 2:
            selected_nodes.append(k)
    try:
         selected_nodes.remove(-1) or selected_nodes.remove(0)
        
    except: 
          pass
    return(selected_nodes)

def n_th_neighbors(graph,node,limit):
    spatial_dict = nx.single_source_shortest_path_length(graph, node, cutoff=limit)
    return spatial_dict

In [11]:
neighbor_dict = {}
cell_type_per_node = {}

for g in range(len(v)):
    neighbor_dict.update({g:get_nodes(n_th_neighbors(gr,g,2))})
    
for g in range(len(v)):
    neigh = get_cell_type(get_nodes(n_th_neighbors(gr,node = g,limit = 2)))
    sub_df = neigh['type'].value_counts(normalize=True)
    cell_type_per_node.update({g:sub_df})
   
with open(snakemake.output[0], 'wb') as f:
     pickle.dump(neighbor_dict, f)   

with open(snakemake.output[1], 'wb') as f:
    pickle.dump(cell_type_per_node, f)  

In [12]:
cell_type_per_node[20]

Excitatory neurons    0.6
Astrocytes            0.2
Oligodendrocytes      0.2
Name: type, dtype: float64

In [13]:
import numpy as np
import skimage
import pandas as pd
import anndata as ad
from collections import namedtuple
import networkx as nx
import math
import sys
import getopt
import anndata as ad
from voronoi import *
import pickle
from pathlib import Path


adata = ad.read(Path(snakemake.input[0]))
x_coord = adata.X[:,-2]
y_coord = adata.X[:,-1]

Point = namedtuple('Point', ['x','y'])
point_list = []
for i,y in zip(x_coord,y_coord):
    point_list.append(Point(i,y))
v,l,e = computeVoronoiDiagram(point_list)

#Building Graph
gr = nx.Graph()
for pt in v:
    gr.add_node(pt)
    
edge_list= []
for ei in range(len(e)):
    edge_list.append(e[ei][1:])
    
for ed in edge_list:
    gr.add_edge(ed[0],ed[1])
    
df = pd.read_csv(Path(snakemake.input[1]))
cell_type = {}
for i,ty in zip(df.id,df.type):
    cell_type.update({i: ty})
    
 
def get_neighbors(graph, node_val, rad):
    subgraph = nx.ego_graph(G = graph, n= node_val, radius=rad, center=True, undirected=False, distance=None)
    node_list = list(subgraph.nodes())
    return(node_list)
    

def get_cell_type(node_list):
    cell_frame = pd.DataFrame()
    cell_list = []
    final_node_list = []
    try:
        node_list.remove(0)
    except: 
        pass
    try:
        node_list.remove(-1)
    except:
        pass
    for n in node_list:
        try:
            cell_list.append(cell_type[n])
            final_node_list.append(n)
            
        except:
            node_list.remove(n)
            pass
    
    cell_frame['id'] = final_node_list
    cell_frame['type'] = cell_list
   
    return cell_frame 
        

def cell_pixel_intensity_gfap(frame):
    intensity_list = []
    for idx in frame.id:
        intensity_list.append(mean_intensities[idx - 1]) #minus 1 as we removed intensity for label 0
        
    frame['gfap_intensity'] = intensity_list
    
    
    return frame

def get_nodes(dt):
    selected_nodes = []
    for k in dt.keys():
        if dt[k] == 2:
            selected_nodes.append(k)
    try:
         selected_nodes.remove(-1) or selected_nodes.remove(0)
        
    except: 
          pass
    return(selected_nodes)

def n_th_neighbors(graph,node,limit):
    spatial_dict = nx.single_source_shortest_path_length(graph, node, cutoff=limit)
    return spatial_dict

neighbor_dict = {}
cell_type_per_node = {}

for g in range(len(v)):
    neighbor_dict.update({g:get_nodes(n_th_neighbors(gr,g,2))})
    
for g in range(len(v)):
    neigh = get_cell_type(get_nodes(n_th_neighbors(gr,node = g,limit = 2)))
    sub_df = neigh['type'].value_counts(normalize=True)
    cell_type_per_node.update({g:sub_df})
   
with open(snakemake.output[0], 'wb') as f:
     pickle.dump(neighbor_dict, f)   

with open(snakemake.output[1], 'wb') as f:
    pickle.dump(cell_type_per_node, f)  
        
        
    


/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [14]:
cell_type_per_node[20]

Excitatory neurons    0.6
Astrocytes            0.2
Oligodendrocytes      0.2
Name: type, dtype: float64

{1: 'Oligodendrocytes',
 2: 'Oligodendrocytes',
 3: 'Inhibitory neurons',
 4: 'Excitatory neurons',
 5: 'Cholinergic neurons',
 6: 'Oligodendrocytes',
 7: 'Excitatory neurons',
 8: 'Excitatory neurons',
 9: 'Oligodendrocytes',
 10: 'Oligodendrocytes',
 11: 'Cholinergic neurons',
 12: 'Oligodendrocytes',
 13: 'Oligodendrocytes',
 14: 'Oligodendrocytes',
 15: 'Oligodendrocytes',
 16: 'Oligodendrocytes',
 17: 'Endothelial cells',
 18: 'Oligodendrocytes',
 19: 'Inhibitory neurons',
 20: 'Oligodendrocytes',
 21: 'Oligodendrocytes',
 22: 'Microglia/Macrophages',
 23: 'Excitatory neurons',
 24: 'Cholinergic neurons',
 25: 'Oligodendrocytes',
 26: 'Cholinergic neurons',
 27: 'Oligodendrocytes',
 28: 'Oligodendrocytes',
 29: 'Endothelial cells',
 30: 'Oligodendrocytes',
 31: 'Inhibitory neurons',
 32: 'Inhibitory neurons',
 33: 'Excitatory neurons',
 34: 'Oligodendrocytes',
 35: 'Astrocytes',
 36: 'Inhibitory neurons',
 37: 'Astrocytes',
 38: 'Oligodendrocytes',
 39: 'Oligodendrocytes',
 40: 'A

In [1]:
cd /gpfs/commons/groups/nygcfaculty/PySeq/20210323_4i4color/pipeline/20210323_4i4color/

/gpfs/commons/groups/nygcfaculty/PySeq/20210323_4i4color/pipeline/20210323_4i4color


In [2]:
ls

anndata/            masks/             summary_m3b.yaml  test_directory3/
cell_neighborhood/  net_neighborhood/  tables/           test_directory4/
final_zarr/         processed_zarr/    test/             test_directory5/
intensities/        raw_zarr/          test_directory1/
logs/               summary_m1a.yaml   test_directory2/


In [3]:
cd intensities

/gpfs/commons/groups/nygcfaculty/PySeq/20210323_4i4color/pipeline/20210323_4i4color/intensities


In [4]:
ls

m1a.pkl  m3b.pkl


In [12]:
import pickle
file = open("m3b.pkl",'rb')
object_file = pickle.load(file)

In [13]:
object_file['LMN1b']

[374.4911504424779,
 564.6321839080459,
 360.57312252964425,
 844.3019390581718,
 489.53164556962025,
 256.3409490333919,
 827.4922680412371,
 558.7714285714286,
 196.13425345043916,
 640.3063063063063,
 432.93333333333334,
 619.1734693877551,
 1474.9339019189765,
 538.0578512396694,
 578.85,
 854.8161764705883,
 981.9197860962566,
 371.9636135508156,
 870.8214285714286,
 1242.6743421052631,
 715.9825327510918,
 349.4923076923077,
 962.6866666666666,
 620.7317073170732,
 568.9527559055118,
 582.6921487603306,
 595.6602564102565,
 430.061797752809,
 1295.9497206703911,
 538.24,
 537.2085889570552,
 718.5103734439834,
 544.6906474820144,
 410.08626198083067,
 580.77734375,
 1046.9585798816568,
 805.5799151343706,
 371.06802721088434,
 775.2475247524752,
 532.1111111111111,
 580.0938775510205,
 476.1604938271605,
 917.8157894736842,
 944.120253164557,
 628.2235294117647,
 1014.7692307692307,
 782.0804020100503,
 997.5144927536232,
 624.0845295055822,
 1166.3594306049822,
 1115.81553398058

In [14]:
object_file

{'LMN1b': [374.4911504424779,
  564.6321839080459,
  360.57312252964425,
  844.3019390581718,
  489.53164556962025,
  256.3409490333919,
  827.4922680412371,
  558.7714285714286,
  196.13425345043916,
  640.3063063063063,
  432.93333333333334,
  619.1734693877551,
  1474.9339019189765,
  538.0578512396694,
  578.85,
  854.8161764705883,
  981.9197860962566,
  371.9636135508156,
  870.8214285714286,
  1242.6743421052631,
  715.9825327510918,
  349.4923076923077,
  962.6866666666666,
  620.7317073170732,
  568.9527559055118,
  582.6921487603306,
  595.6602564102565,
  430.061797752809,
  1295.9497206703911,
  538.24,
  537.2085889570552,
  718.5103734439834,
  544.6906474820144,
  410.08626198083067,
  580.77734375,
  1046.9585798816568,
  805.5799151343706,
  371.06802721088434,
  775.2475247524752,
  532.1111111111111,
  580.0938775510205,
  476.1604938271605,
  917.8157894736842,
  944.120253164557,
  628.2235294117647,
  1014.7692307692307,
  782.0804020100503,
  997.5144927536232,
 